In [1]:
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd

from utils.dataloader import create_dataloader, create_dataset
from utils.utils import sorted_file_paths, find_latest_checkpoint

from models.resnet import load_resnet_model, register_model_with_hook

from visualization.lots import LOTS, calculate_activation_map

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

category_names = [
    "water", "trees", "grass", "flooded_vegetation", "crops",
    "shrub_and_scrub", "built", "bare", "snow_and_ice"
]

experiments = [
    ['L2', './experiments/ex_2'],
    ['Weighted sampling (alpha=0.5)', './experiments/ex_8'],
]

In [14]:
def calculate_activation_df(model, get_feature_maps, category_names, loader, device):
    # Initialize storage structures
    category_areas = {name: [] for name in category_names}
    average_intensities = {name: [] for name in category_names}
    all_labels = []
    all_predictions = []
    all_locations = []

    for data, location, labels in tqdm(loader):
        with torch.no_grad():
            data = data.to(device)
            labels = labels.to(device)
            predictions = model(data[:, :24, :, :])

        # Flatten labels and predictions and store
        all_labels.extend(labels.cpu().numpy().flatten())
        all_locations.extend([f"{row[0]} + {row[1]}" for row in location.cpu().numpy()])
        all_predictions.extend(predictions.cpu().numpy().flatten())

        for i in range(data.size(0)):
            # Generate initial and adversarial images
            imageinit = data[i, :24, :, :].unsqueeze(0)
            imageadv,_ = LOTS(imageinit, 50, model, get_feature_maps, device, alpha=1)

            # Calculate the activation map without normalization
            activation_map = calculate_activation_map(imageinit.squeeze(), imageadv.squeeze(), filter_size=5, normalize=False)

            # Process each category within the image
            land_cover_mask = data[i, 24, :, :]  # Assuming the land cover mask is at channel index 24
            for idx, category in enumerate(category_names):
                category_mask = (land_cover_mask == idx).float()
                category_area = category_mask.sum().item()
                category_areas[category].append(category_area)

                if category_mask.sum() > 0:
                    average_intensity = (activation_map * category_mask).sum() / category_mask.sum()
                    average_intensities[category].append(average_intensity.item())
                else:
                    average_intensities[category].append(np.nan)  # Handle no area case

    # Convert results to DataFrames for better manipulation and visibility
    intensities_df = pd.DataFrame(average_intensities)
    areas_df = pd.DataFrame(category_areas)

    # Concatenate the areas DataFrame with the intensities DataFrame
    result_df = pd.concat([intensities_df, areas_df.add_suffix("_area")], axis=1)

    # Add location, label, and prediction data
    result_df['Location'] = all_locations
    result_df['Label'] = all_labels
    result_df['Prediction'] = all_predictions

    return result_df

In [3]:
test_files = sorted_file_paths('./data/test_records_with_land_cover/test')
test_dataset = create_dataset(test_files, half= False)
test_loader = create_dataloader(test_dataset, 64)

In [15]:
for experiment, path in experiments:
    model = load_resnet_model()
    checkpoint = find_latest_checkpoint(path)
    print(checkpoint)
    statedict = torch.load(checkpoint)
    model.load_state_dict(statedict)
    model.eval()
    model = model.to(device)
    get_feature_maps = register_model_with_hook(model)
    df = calculate_activation_df(model, get_feature_maps, category_names, test_loader, device)
    df.to_csv(f'./visualization_validation/{experiment}.csv', index=False)

./experiments/ex_2\checkpoint_epoch_6.pth


100%|██████████| 33/33 [50:54<00:00, 92.57s/it]


./experiments/ex_8\checkpoint_epoch_17.pth


100%|██████████| 33/33 [50:36<00:00, 92.03s/it]
